<a href="https://colab.research.google.com/github/MoosaSaadat/handsonml/blob/master/style_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model

import numpy as np
import matplotlib.pyplot as plt

tf.executing_eagerly()

True

In [2]:
model = VGG19(
    include_top=False,
    weights="imagenet"
)

model.trainable = False
model.summary()

80142336/80134624 [==============================] - 2s 0us/step
Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
______________________________________________________________

In [3]:
# Utility functions
def load_and_process_image(img_path):
  img = load_img(img_path)
  img = img_to_array(img)
  img = preprocess_input(img)
  img = np.expand_dims(img, axis=0)
  return img

def deprocess(x):
  x[:,:,0] += 103.939
  x[:,:,1] += 116.779
  x[:,:,2] += 123.680
  x = x[:,:,::-1]
  x = np.clip(x, 0, 255).astype('uint8')
  return x

def display_image(image):
  if len(image.shape) == 4:
    img = np.squeeze(image, axis=0)

  img = deprocess(img)
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  plt.imshow(img)

In [6]:
content_layer = "block5_conv2"
style_layers = [
    "block1_conv1",
    "block3_conv1",
    "block5_conv1"
]

content_model = Model(
    inputs = model.input,
    outputs = model.get_layer(content_layer).output
)

style_models = [Model(
    inputs = model.input,
    outputs = model.get_layer(style_layer). output
) for style_layer in style_layers]

In [7]:
def content_cost(content, generated):
  a_C = content_model(content)
  a_G = content_model(generated)
  cost = tf.reduce_mean(tf.square(a_C - a_G))
  return cost